## Item a) 

Considere a Equação de Euler (2).



In [2]:
pip install fredapi

In [3]:
from fredapi import Fred
from datetime import datetime
from scipy import stats
from statsmodels.sandbox.regression import gmm
from google.colab import files

import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
#######################################
#### Importando os dados do Github ####
#######################################


# Importando os dados do Github
df_q  = pd.read_csv('https://raw.githubusercontent.com/lucasestrela/dec_consumo_poupanca/main/dados/BEA_Tab115_Quarter.csv',
                             delimiter = ";")

p_q  = pd.read_csv('https://raw.githubusercontent.com/lucasestrela/dec_consumo_poupanca/main/dados/BEA_Tab114_Quarter.csv',
                             delimiter = ";")

# Deletando a primeira coluna
del df_q['Line']
del p_q['Line']

# Transpondo a matriz
df_q  = df_q.T
p_q   = p_q.T

# Excluindo primeira linha a add ela como var name
df_q.columns = df_q.iloc[0, :]
p_q.columns  = p_q.iloc[0, :]

df_q = df_q.iloc[1:, :]
p_q  = p_q.iloc[1:, :]



# Removendo o underline
df_q.index = df_q.index.str.replace("_", "")
p_q.index = p_q.index.str.replace("_", "")


# Transformando em data
df_q.index = pd.to_datetime( df_q.index , errors='coerce')
p_q.index = pd.to_datetime( p_q.index , errors='coerce')

# Renomeando
df_q = df_q.rename(columns={df_q.columns[2]: 'bens', df_q.columns[3]: 'duraveis', df_q.columns[4]: 'n_duraveis', df_q.columns[5]: 'servicos'})
p_q  = p_q.rename(columns={p_q.columns[2]: 'bens', p_q.columns[3]: 'duraveis', p_q.columns[4]: 'n_duraveis', p_q.columns[5]: 'servicos'})

# Transformando em .float
df_q = df_q[['bens', 'duraveis', 'n_duraveis', 'servicos']].apply(pd.to_numeric, errors='coerce')
p_q = p_q[['bens', 'duraveis', 'n_duraveis', 'servicos']].apply(pd.to_numeric, errors='coerce')

# Calculando consumo real, separando duráveis e não duraveis NAO TEM SERVICO
duraveis_real = df_q['duraveis']/p_q['duraveis']
consumo_real = df_q['n_duraveis']/p_q['n_duraveis']

# Crescimento real do consumo
cresc_duravel_real = duraveis_real/duraveis_real.shift(1) - 1
cresc_consumo_real = consumo_real/consumo_real.shift(1) - 1

# Calculando a tx de inflação (1+...%) = taxa de inflação de todos os bens na economia
tx_inf1 = p_q['bens']/p_q['bens'].shift(1)


dta = pd.DataFrame([consumo_real, cresc_consumo_real,  p_q['n_duraveis'], duraveis_real, cresc_duravel_real, p_q['duraveis'], tx_inf1]).T
dta.columns = ['c', 'c_growth', 'pc', 'k', 'k_growth', 'pk', 'inflacao']

In [5]:
dta

,c,c_growth,pc,k,k_growth,pk,inflacao
1947-01-01,5157.184185,NaN,14.518,445.634409,NaN,46.500,NaN
1947-04-01,5245.719353,0.017167,14.659,454.973577,0.020957,46.928,1.009585
1947-07-01,5269.967837,0.004623,14.924,461.326552,0.013963,47.190,1.015302
1947-10-01,5200.975610,-0.013092,15.375,491.329359,0.065036,47.805,1.026329
1948-01-01,5218.276061,0.003326,15.627,488.831223,-0.005084,48.170,1.014375
...,...,...,...,...,...,...,...
2019-10-01,30181.122270,-0.001879,99.673,18118.905496,0.007707,85.774,1.000538
2020-01-01,30704.753677,0.017350,99.544,17521.532445,-0.032970,85.406,0.997701
2020-04-01,29478.355554,-0.039942,97.808,17447.791141,-0.004209,84.727,0.985719
2020-07-01,31543.796766,0.070066,98.763,20284.656330,0.162592,86.420,1.013309


In [6]:
#######################################
###### Importando dados do SP500 ######
#######################################


sp500  = pd.read_csv('https://raw.githubusercontent.com/lucasestrela/dec_consumo_poupanca/main/dados/sp500.csv',
                             delimiter = ",")

# Selecionando variaveis
sp500 = sp500[['ref.date', 'price.adjusted']]
sp500.columns = ['date', 'price']

# Adicionando o indice de datas
sp500.index = pd.to_datetime( sp500['date'] , errors='coerce')


# Criando variavel indicadora para mudança de quarter
sp500['quarter'] = sp500.index.quarter
sp500['year'] = sp500.index.year
sp500['group_indicator'] = sp500['quarter']  != sp500['quarter'] .shift(-1)

# Filtrando apenas os ultimos meses do quarter e selecionando ano que quero
sp500 = sp500[sp500['group_indicator'] == True]
sp500 = sp500[sp500['year'] >= 1947][:-2]

# Selecionando variaveis
sp500 = sp500[['price']]


# Copiando o indice
sp500.index = p_q.index

# Calculando o retonro e o ret real
sp500['sp500_ret_real'] = ((sp500['price']/sp500['price'].shift(1)) / tx_inf1) - 1
sp500['sp500_ret'] = sp500['price']/sp500['price'].shift(1) - 1

# Fazendo o merge dos dois datasets
dta = dta.join(sp500)

In [7]:
fred = Fred(api_key='548624ed839059b759900a51992bc8c8')

# Obtendo dados trimestrais do FRED usando a média do período
tb3m = fred.get_series('DTB3', frequency = 'q', aggregation_method = 'avg')

# Transformando em dataframe
tb3m = pd.DataFrame(tb3m, columns = ['T-Bill'])

# Transformando o retorno em trimestral
tb3m['TBill_ret'] = np.power(1+tb3m['T-Bill']/100, 0.25) - 1

In [8]:
# Fazendo o merge dos dois datasets
dta2 = dta.join(tb3m['TBill_ret'])

dta2['TBill_ret_real'] = ( 1 + dta2['TBill_ret'] ) / dta2['inflacao'] - 1

dta2 = dta2[['c', 'c_growth', 'pc', 'k', 'k_growth', 'pk', 'sp500_ret_real', 'TBill_ret_real']].dropna()

In [9]:
dta2.head()

,c,c_growth,pc,k,k_growth,pk,sp500_ret_real,TBill_ret_real
1954-01-01,6032.108179,0.009646,16.787,612.575385,-0.030679,53.724,0.054588,-0.001841
1954-04-01,6016.150101,-0.002646,16.842,642.405728,0.048697,52.375,0.116224,0.005776
1954-07-01,6092.160954,0.012634,16.775,645.100551,0.004195,51.516,0.036534,0.009413
1954-10-01,6194.585854,0.016813,16.697,684.643171,0.061297,51.215,0.137278,0.007692
1955-01-01,6259.103977,0.010415,16.696,745.842631,0.089389,51.535,0.082005,0.001627


In [10]:
# Exportanto dados
# dta2.to_csv('dataset.csv')
# files.download('dataset.csv')